# SQL Cheat Sheet

In [1]:
# !pip install ipython-sql
# !pip install psycopg2  # PostgreSQL
# !pip install mysql  # MySQL

In [2]:
SQL_CONN = 'postgresql://wing@localhost/wing'

%config SqlMagic.displaycon=False
%load_ext sql
%sql {SQL_CONN}

## Tables & Types

In [3]:
%sql CREATE TYPE t_grade AS ENUM('A', 'B', 'C', 'D');

(psycopg2.errors.DuplicateObject) type "t_grade" already exists

[SQL: CREATE TYPE t_grade AS ENUM('A', 'B' , 'C' , 'D' );]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [4]:
%%sql
CREATE TABLE advisors (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL
);

CREATE TABLE students (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    age INT NOT NULL,
    height DECIMAL(8,2),
    grade t_grade,
    advisorId INT NOT NULL,
    created TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY(advisorId) REFERENCES advisors(id)
);

Done.
Done.


[]

In [5]:
%%sql
ALTER TABLE advisors
ADD telephone VARCHAR(255);

ALTER TABLE advisors
DROP COLUMN telephone;

Done.
Done.


[]

In [6]:
%%sql
DROP TABLE students;
DROP TABLE advisors;

Done.
Done.


[]

## Views

In [7]:
%%sql
CREATE VIEW cust_names AS
SELECT firstName, lastName FROM customers;

SELECT * from cust_names

Done.
4 rows affected.


firstname,lastname
jane,doe
john,doe
jane,austen
jack,daniels


In [8]:
%%sql
DROP VIEW cust_names

Done.


[]

## Records

In [9]:
%%sql
INSERT INTO customers (firstName, lastName, age) VALUES
('jack', 'daniels', 99);

1 rows affected.


[]

In [10]:
%%sql
UPDATE customers SET
email='jack@daniels.net',
state='CA'
WHERE lastname='daniels' AND age=99;

1 rows affected.


[]

In [11]:
%%sql
UPDATE customers SET
age=age-1
WHERE lastname='daniels' AND age=99;

1 rows affected.


[]

In [12]:
%%sql
DELETE FROM customers
WHERE firstName='jack' AND age=98;

1 rows affected.


[]

## PROCEDURE

In [13]:
%%sql
CREATE PROCEDURE insert_delete(first_name VARCHAR, last_name VARCHAR, age INT)
BEGIN ATOMIC
INSERT INTO customers (firstName, lastName, age) VALUES
(first_name, last_name, age);
DELETE FROM customers
WHERE firstName=first_name and lastName=last_name;
END;

Done.


[]

In [14]:
%%sql
CALL insert_delete('joe', 'shmoe', 2);
DROP PROCEDURE insert_delete;

Done.
Done.


[]

## SELECT

SELECT ... (AS ...) FROM ... (AS ...) WHERE ... GROUP BY ... HAVING ... ORDER BY ...

In [15]:
%%sql
SELECT * from customers ORDER BY age DESC;

4 rows affected.


id,firstname,lastname,age,email,state
5,jane,austen,22,None,None
4,jack,daniels,20,None,WA
2,john,doe,19,john@doe.com,OR
1,jane,doe,18,jane@doe.com,OR


In [16]:
%%sql
SELECT CONCAT(firstName, ' ', lastName) AS name, age + 1 AS age_plus_1 from customers;

4 rows affected.


name,age_plus_1
jane doe,19
john doe,20
jane austen,23
jack daniels,21


In [17]:
%%sql
SELECT MIN(age) from customers; -- also: MAX()

1 rows affected.


min
18


In [18]:
%%sql
SELECT AVG(age) from customers; -- also: SUM()

1 rows affected.


avg
19.7500000000000000


In [19]:
%%sql
SELECT COUNT(*) from customers where age>20;

1 rows affected.


count
1


## GROUP BY  &  HAVING

In [20]:
%sql SELECT lastName, COUNT(lastName) FROM customers GROUP BY lastName;

3 rows affected.


lastname,count
daniels,1
austen,1
doe,2


In [21]:
%sql SELECT lastName, COUNT(lastName) FROM customers GROUP BY lastName HAVING COUNT(lastName)>1;

1 rows affected.


lastname,count
doe,2


## WHERE

In [22]:
%sql SELECT * from customers WHERE firstname='john' AND lastname='doe';

1 rows affected.


id,firstname,lastname,age,email,state
2,john,doe,19,john@doe.com,OR


In [23]:
%sql SELECT * from customers WHERE age>=20;

2 rows affected.


id,firstname,lastname,age,email,state
5,jane,austen,22,None,None
4,jack,daniels,20,None,WA


In [24]:
%sql SELECT * from customers WHERE age BETWEEN 17 AND 19

2 rows affected.


id,firstname,lastname,age,email,state
1,jane,doe,18,jane@doe.com,OR
2,john,doe,19,john@doe.com,OR


In [25]:
%sql SELECT * from customers WHERE firstName LIKE 'jan%';

2 rows affected.


id,firstname,lastname,age,email,state
1,jane,doe,18,jane@doe.com,OR
5,jane,austen,22,None,None


In [26]:
%sql SELECT * from customers WHERE firstName NOT LIKE 'ja__';

1 rows affected.


id,firstname,lastname,age,email,state
2,john,doe,19,john@doe.com,OR


In [27]:
%sql SELECT * from customers WHERE lastName IN ('austen', 'daniels');

2 rows affected.


id,firstname,lastname,age,email,state
5,jane,austen,22,None,None
4,jack,daniels,20,None,WA


In [28]:
%sql SELECT * from customers WHERE email IS NULL;
# "= NULL" wouldn't work!

2 rows affected.


id,firstname,lastname,age,email,state
5,jane,austen,22,None,None
4,jack,daniels,20,None,WA


## JOIN

In [29]:
%sql SELECT * FROM customers

4 rows affected.


id,firstname,lastname,age,email,state
1,jane,doe,18,jane@doe.com,OR
2,john,doe,19,john@doe.com,OR
5,jane,austen,22,None,None
4,jack,daniels,20,None,WA


In [30]:
%sql SELECT * FROM products

3 rows affected.


id,name,price
1,xbox,299.95
2,ps5,399.00
3,ps6,None


In [31]:
%sql SELECT * FROM orders

3 rows affected.


id,custid,prodid,qty,date
1,1,2,1,2020-01-01 00:00:00
2,2,1,1,2020-01-02 00:00:00
3,4,1,2,2020-02-03 00:00:00


In [32]:
%%sql
SELECT customers.firstName, customers.lastName, products.name AS "Product Name", orders.qty
FROM customers
JOIN orders ON customers.id=orders.custId
JOIN products ON orders.prodId=products.id;

3 rows affected.


firstname,lastname,Product Name,qty
jane,doe,ps5,1
john,doe,xbox,1
jack,daniels,xbox,2


In [33]:
%%sql
SELECT c.firstName, c.lastName, o.id
FROM customers AS c
JOIN orders AS o ON c.id=o.custId;

3 rows affected.


firstname,lastname,id
jane,doe,1
john,doe,2
jack,daniels,3


In [34]:
%%sql
SELECT customers.id, customers.state, states.name
FROM customers
JOIN states ON customers.state=states.state;

2 rows affected.


id,state,name
2,OR,Oregon
1,OR,Oregon


In [35]:
%%sql
SELECT customers.id, customers.state, states.name
FROM customers
LEFT JOIN states ON customers.state=states.state;

4 rows affected.


id,state,name
2,OR,Oregon
1,OR,Oregon
5,None,None
4,WA,None


In [36]:
%%sql
SELECT customers.id, customers.state, states.name
FROM customers
RIGHT JOIN states ON customers.state=states.state;

3 rows affected.


id,state,name
2,OR,Oregon
1,OR,Oregon
None,None,California
